In [ ]:
import cv2 # Importing cv2 library
import numpy as np # Importing numpy library
from time import sleep
import csv

with open("rubikThreshold.csv", 'r') as infile:
    '''
    number	label
    0	green
    1	white
    2	yellow
    3	red
    4	orange
    5	blue
    '''
    thresholds = list(csv.reader(infile))  # 6x6 array, each row contains 1 color, hsv min & max
    thresholds = np.asarray(thresholds, dtype=int)


cap = cv2.VideoCapture(1) #Choose to capture laptopcam or external webcam

#write output of video to a videofile
out = cv2.VideoWriter('filename.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, (640, 480)) 

while True: #continues forever
    retrieveval, frame_org = cap.read()
    h, w, ch = frame_org.shape

    #crop frame by a specific diameter
    d = 360
    frame = frame_org[(h-d)//2:(h+d)//2, (w-d)//2:(w+d)//2]
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) #convert BGR to HSV

    '''RETR_EXTERNAL --> retrieves only the extreme outer contours, concentric circles
    only outermost circle is taken

    RETR_LIST --> gives all the contours, even if found inside one another
    
    RETR_CCOMP --> gives all the contours, outer and inner
    
    RETR_TREE--> retrieves all of the contours and reconstructs a full rlnshp 
    of how all the contours are related to each other
       
    '''

    '''
    CV_CHAIN_APPROX_NONE stores absolutely all the contour points. 
    That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be 
    either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1.

    CV_CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments
    and leaves only their end points. For example, an up-right rectangular 
    contour is encoded with 4 points.

    CV_CHAIN_APPROX_TC89_L1,CV_CHAIN_APPROX_TC89_KCOS applies one of the 
    flavors of the Teh-Chin chain approximation algorithm. 
    
    ''' 
    #iterate through 6 possible types of color
    for i in range(6):
        color = thresholds[i]
        col_type = color[0]
        low = color[1:4]
        high = color[4:]
        if col_type == -2:
            mask = cv2.inRange(frame, low, high)
        else:
            mask = cv2.inRange(hsv_frame, low, high)

        #retrieval is a standard boolean value
        contours, retrieveval = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for con in contours:
            #if contour area is big enough
            if cv2.contourArea(con)>5000:
                (x,y,w,h) = cv2.boundingRect(con)
                
                font                   = cv2.FONT_HERSHEY_SIMPLEX
                bottomLeftCornerOfText = (x+10, y+30)
                fontScale              = 1
                fontColor              = (0,0,0)
                lineType               = 2

                #create text as required
                cv2.putText(frame, str(i), 
                    bottomLeftCornerOfText, 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
        

    #Showing the frame and masks
    cv2.imshow("Tracking1", frame)
    cv2.imshow("Tracking2", frame_org)
    out.write(frame_org)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        cap.release() #release resources
        cv2.destroyAllWindows() #get rid of all live windows
        out.release()
        for i in range(6):
            print(i)
            color = thresholds[i]
            col_type = color[0]
            low = color[1:4]
            high = color[4:]
            print(low, high)
            if col_type == -2:
                mask = cv2.inRange(frame, low, high)
            else:
                mask = cv2.inRange(hsv_frame, low, high)
            contours, retrieveval = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            '''if i==3 or i==4:
                
                print(i)
                contours = sorted(contours, key = lambda x:cv2.contourArea(x), reverse=True)
                for con in contours:
                    print(cv2.contourArea(con))'''
            for con in contours:
                if cv2.contourArea(con)>4500:
                    (x,y,w,h) = cv2.boundingRect(con)
                    print(x,y)
        break

        
        

0
[ 58 141  95] [ 94 255 223]
<class 'numpy.ndarray'>
1
[173 159 183] [255 230 255]
2
[22 81 95] [ 47 255 255]
<class 'numpy.ndarray'>
3
[  7  16 142] [ 81  72 255]
4
[ 33  91 107] [120 180 255]
8 150
5
[ 88 153 132] [122 255 255]
<class 'numpy.ndarray'>


In [ ]:
import cv2 # Importing cv2 library
import numpy as np # Importing numpy library
import csv
import time

def mouseEvent(event, x, y, flags, param):
    global x1, y1
    x1 = x
    y1 = y

cap = cv2.VideoCapture(1) #Choose to capture laptopcam or external webcam
while True: #continues forever
    retrieveval, frame_org = cap.read()
    h, w, ch = frame_org.shape
    d = 360
    frame = frame_org[(h-d)//2:(h+d)//2, (w-d)//2:(w+d)//2]
    frame = cv2.flip(frame, 1)
    cv2.imshow("img", frame)
    cv2.setMouseCallback("img", mouseEvent)
    key = cv2.waitKey(1)
    if key & 0xFF ==ord('k'):
        b,g,r = frame[y1][x1]
        arr = [f"{x:.2f}" for x in [b,g,r,b/g,b/r,g/r,int(b)+int(g)+int(r)]]
        print(' '.join(arr))
        continue
    elif key & 0xFF == ord('t'):
        x,y = input().split(' ')
        b,g,r = frame[int(y)][int(x)]
        arr = [f"{x:.2f}" for x in [b,g,r,b/g,b/r,g/r,int(b)+int(g)+int(r)]]
        print(','.join(arr))
    elif key & 0xFF ==ord('q'):
        cap.release() #release resources
        cv2.destroyAllWindows() #get rid of all live windows
        break
        
        

42.00 92.00 33.00 0.46 1.27 2.79 167.00
42.00 93.00 27.00 0.45 1.56 3.44 162.00
33.00 94.00 27.00 0.35 1.22 3.48 154.00
42.00 95.00 24.00 0.44 1.75 3.96 161.00
41.00 93.00 23.00 0.44 1.78 4.04 157.00
41.00 93.00 23.00 0.44 1.78 4.04 157.00
38.00 89.00 26.00 0.43 1.46 3.42 153.00
40.00 87.00 23.00 0.46 1.74 3.78 150.00
40.00 90.00 22.00 0.44 1.82 4.09 152.00
107.00 68.00 29.00 1.57 3.69 2.34 204.00
111.00 70.00 22.00 1.59 5.05 3.18 203.00
110.00 66.00 18.00 1.67 6.11 3.67 194.00
114.00 68.00 27.00 1.68 4.22 2.52 209.00
111.00 70.00 28.00 1.59 3.96 2.50 209.00
105.00 71.00 31.00 1.48 3.39 2.29 207.00
44.00 80.00 254.00 0.55 0.17 0.31 378.00
43.00 79.00 253.00 0.54 0.17 0.31 375.00
52.00 77.00 255.00 0.68 0.20 0.30 384.00
49.00 83.00 253.00 0.59 0.19 0.33 385.00
42.00 76.00 255.00 0.55 0.16 0.30 373.00
134.00 153.00 167.00 0.88 0.80 0.92 454.00
140.00 154.00 162.00 0.91 0.86 0.95 456.00
138.00 152.00 169.00 0.91 0.82 0.90 459.00
147.00 160.00 174.00 0.92 0.84 0.92 481.00
21.00 29.00 103.0

In [ ]:
a = 1/3
print(f"{a:.2f}")

0.33


In [ ]:
'''
green b/g<0.65 && g/r>2
white 0.9<*<1.1
yellow b/g<0.65 && g/r<1
red b/r<0.3
orange b/r>0.6
blue b/r>1.8

'''

'\ngreen b/g<0.65 && g/r>2\nwhite 0.9<*<1.1\nyellow b/g<0.65 && g/r<1\nred b/r<0.3\norange b/r>0.6\nblue b/g>1.1\n\n'

In [ ]:
import cv2 # Importing cv2 library
import numpy as np # Importing numpy library
import csv
import time
import os

def mouseEvent(event, x, y, flags, param):
    global x1, y1
    x1 = x
    y1 = y

cap = cv2.VideoCapture(1) #Choose to capture laptopcam or external webcam
while True: #continues forever
    retrieveval, frame_org = cap.read()
    #frame_org = cv2.flip(frame_org, 1)
    h, w, ch = frame_org.shape
    d = 360
    frame = cv2.bitwise_or(np.zeros(frame_org.shape, np.uint8), frame_org)
    frame = frame[(h-d)//2:(h+d)//2, (w-d)//2:(w+d)//2]
    
    height, width, channels = frame.shape
    step = 2
    masks = [np.zeros((height//step, width//step), np.uint8) for i in range(6)]
    for i in range(0, len(frame), step):
        for j in range(0, len(frame[i]), step):
            b,g,r = list(map(float, frame[i][j]))
            if b > 0.9*r and g > 1.5*r:  # green and blue
                if b < g:  # green
                    masks[0][i//step][j//step] = 255
                else:  # blue
                    masks[5][i//step][j//step] = 255
            elif b+g+r<150:  # black
                pass
            elif g**2>1.5*b*r and g>1.5*b:  #yellow
                masks[2][i//step][j//step] = 255
            elif 0.6*g<b<1.2*g and 0.6*r<b<1.2*r and 0.6*r<g<1.2*r and b+g+r>200:  # white
                masks[1][i//step][j//step] = 255 
            elif b+g+r<300:  # red
                masks[3][i//step][j//step] = 255
            else:
                masks[4][i//step][j//step] = 255
    notable = []
    for i in range(len(masks)):
        continue
        contours, retrieveval = cv2.findContours(masks[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for con in contours:
            if cv2.contourArea(con)>1600:
                (x,y,w,h) = cv2.boundingRect(con)
                font                   = cv2.FONT_HERSHEY_SIMPLEX
                bottomLeftCornerOfText = (x*step+10, y*step+30)
                fontScale              = 1
                fontColor              = (0,0,0)
                lineType               = 2
                cv2.putText(frame, str(i), 
                    bottomLeftCornerOfText, 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
                notable.append((x*step//(width//4), y*step//(height//4), i))
        cv2.imshow(str(i), masks[i])
    
    cv2.imshow("cropped", frame)
    cv2.imshow("fullscreen", frame_org)
    cv2.setMouseCallback("cropped", mouseEvent)
    
    key = cv2.waitKey(1)
    if key & 0xFF ==ord('k'):
        pass
        for i in range(len(masks)):
            contours, retrieveval = cv2.findContours(masks[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key = lambda x:cv2.contourArea(x), reverse=True)
            areas = [cv2.contourArea(x) for x in contours]
            print(areas[:5])
    elif key & 0xFF == ord('t'):
        b,g,r = frame[y1][x1]
        arr = [f"{x:.2f}" for x in [b,g,r,b/g,b/r,g/r,int(b)+int(g)+int(r)]]
        print(' '.join(arr))
    elif key & 0xFF ==ord('q'):
        cap.release() #release resources
        cv2.destroyAllWindows() #get rid of all live windows
        break
    elif key & 0xFF == ord('b'):
        path = "./images2/"
        if len(notable)!=9:
            continue
        answers = sorted(notable,key=lambda x: (x[1],x[0]))
        labels = ','.join([str(x[2]) for x in answers])
        maxnum = -1
        for item in os.listdir(path):
            if labels in item:
                maxnum = max(maxnum, int(item[18:-4]))
        cv2.imwrite(path+labels+f'_{maxnum+1}.png', frame_org)

In [ ]:
import os
k = os.listdir('images2/')